In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix
from contemporary_ner_training.conll_ner_importer import conll_to_ner_labelling  
from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

from estnltk.taggers import VabamorfCorpusTagger
vm_corpus_tagger = VabamorfCorpusTagger()

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

CONLL NER test:

In [ ]:
'''
texts = []
for file in os.listdir('contemporary_ner_training'):
    if file.startswith('estner_split_1'):
        texts.append(conll_to_ner_labelling(os.path.join('contemporary_ner_training', file), 'gold_wordner'))               
'''

Flags:

In [3]:
use_vabamorfcorpustagger = False

---

In [4]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [5]:
def create_training_texts(filenames):
    print("(!) Valmistan ette treenimistekste")
    
    # These files don't work because the protocols are written in a different language,
    # which the goldstandard didn't recognise, hence have no goldstandard tags.
    files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                         'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                         'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                         'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                         'P2rnu_Halliste_Abja_id257_1844a.json', \
                         'Saare_Kaarma_Loona_id7575_1899a.json', \
                         'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                         'J2rva_Tyri_V22tsa_id22178_1912a.json']
    
    start = time.time()
    training_texts = []
    for filename in filenames:
        with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
            if filename in files_not_working:
                continue
            else:
                tagged_text = preprocess_text(json_to_text(file.read()))
                if use_vabamorfcorpustagger:
                    tagged_text.pop_layer('morph_analysis')
                    vm_corpus_tagger.tag([tagged_text])
                training_texts.append(tagged_text)
    print(f"(!) Treenimistekstid ette valmistatud {time.time() - start} sekundiga")
    return training_texts

In [6]:
def train_nertagger(training_texts, new_model_dir):
    print("(!) Treenin NerTaggerit")
    start = time.time()
    
    modelUtil = ModelStorageUtil( new_model_dir )
    nersettings = modelUtil.load_settings()
    trainer = NerTrainer(nersettings)
    trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
    print(f"(!) NerTagger treenitud {time.time() - start} sekundiga")

In [7]:
def get_testing_and_training_subdistribution(subdistribution):
    training = []
    for y in [1, 2, 3, 4, 5]:
        if y == subdistribution:
            testing = y
        else:
            training.append(y)
    return testing, training

In [22]:
def tag_files(model_dir, testing_files, use_vabamorfcorpustagger):
    removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']
    nertagger = NerTagger(model_dir)
    
    print("(!) Märgendan")
    start = time.time()
    iterator = 1
    for test_file in testing_files:
        with open(find(test_file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
            text = f.read()
        
        if test_file == "Tartu_V6nnu_Ahja_id3502_1882a.json":
            text = text.replace('..', '. .')
        text = preprocess_text(Text(text))

        if use_vabamorfcorpustagger or "vabamorf" in model_dir:
            text.pop_layer('morph_analysis')
            text = [text]
            vm_corpus_tagger.tag( text )
            text = text[0]
        nertagger.tag(text)
        text.add_layer(flatten(text['ner'], 'flat_ner'))

        for x in removed_layers:
            text.pop_layer(x)

        path = os.path.join(model_dir, 'vallakohtufailid-trained-nertagger')
        if not os.path.exists(path):
            os.mkdir(path)
            
        text_to_json(text, file=os.path.join(model_dir, 'vallakohtufailid-trained-nertagger', test_file))
        
        print(f'{iterator}. Märgendatud fail {test_file}')
        iterator += 1
    print(f"(!) Failid märgendatud {time.time() - start} sekundiga")

In [26]:
def train_model(model_directory):
    for subdistribution in [1, 2, 3, 4, 5]:
        testing, training = get_testing_and_training_subdistribution(subdistribution)

        # Get the filenames to be trained on from the files dictionary
        filenames = [key for key, value in files.items() if int(value) in training]

        # Create training_texts from the aforementioned filenames
        training_texts = create_training_texts(filenames)

        # Set up the trainer and training
        new_model_dir = os.path.join('models', model_directory)
        train_nertagger(training_texts, new_model_dir)

        # Set up the new trained nertagger and defining layers to be removed later on
        tagger = NerTagger(model_dir = new_model_dir)
        #print(tagger.nersettings)
        # Tag the files using the new nertagger
        testing_files = [key for key, value in files.items() if int(value) == testing]
        tag_files(new_model_dir, testing_files, use_vabamorfcorpustagger)
            
    # Get results of model
    extract_results_to_txt_file(model_directory, files)
    
    print(f"(!) Mudel {model_directory} treenitud")

To train the model the `model_directory` must contain a `settings.py` file

In [27]:
models = [
    'model_default_with_vabamorftagger',
    'model_local_features_without_morph',
    'model_morph_without_lemmas',
    'model_morph_with_lemmas',
    'model_morph_with_lemmas_and_sentences',
    'model_morph_with_lemmas_and_sentences_and_gazzetteer',
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_initial'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_vabamorf_gazetteer'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_vabamorf_gazetteer2'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_vabamorf_gazetteer1and2'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_gaz_loc'),
    os.path.join('model_morph_with_lemmas_and_sentences_and_gazetteer_and_global_features', 'model_gaz_loc_variants')    
]
for model in models:
    train_model(model)

(!) Valmistan ette treenimistekste
Treenimistekstid ette valmistatud 112.98634362220764 sekundiga.
(!) Treenin NerTaggerit
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 33508
Seconds required: 0.206

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30999.990087
Trial #1 (eta = 0.100000): 3844.097079
Trial #2 (eta = 0.200000): 5400.307219
Trial #3 (eta = 0.400000): 10056.158513
Trial #4 (eta = 0.800000): 17363.458636
Trial #5 (eta = 1.600000): 32051.440575 (worse)
Trial #6 (eta = 0.050000): 3553.442400
Trial #7

***** Epoch #35 *****
Loss: 4004.374920
Improvement ratio: 0.136531
Feature L2-norm: 131.947125
Learning rate (eta): 0.049826
Total number of feature updates: 568505
Seconds required for this iteration: 0.200

***** Epoch #36 *****
Loss: 3832.359310
Improvement ratio: 0.146958
Feature L2-norm: 133.236176
Learning rate (eta): 0.049821
Total number of feature updates: 584748
Seconds required for this iteration: 0.200

***** Epoch #37 *****
Loss: 3819.051097
Improvement ratio: 0.228300
Feature L2-norm: 134.492301
Learning rate (eta): 0.049816
Total number of feature updates: 600991
Seconds required for this iteration: 0.199

***** Epoch #38 *****
Loss: 3720.993324
Improvement ratio: 0.217609
Feature L2-norm: 135.716417
Learning rate (eta): 0.049811
Total number of feature updates: 617234
Seconds required for this iteration: 0.199

***** Epoch #39 *****
Loss: 3808.643347
Improvement ratio: 0.178336
Feature L2-norm: 136.959613
Learning rate (eta): 0.049806
Total number of feature updates: 6

68. Märgendatud fail Harju_Keila_Saue_id10625_1884a.json
69. Märgendatud fail L22ne_Kullamaa_Piirsalu_id12153_1893a.json
70. Märgendatud fail Tartu_R6ngu_Aakre_id5583_1889a.json
71. Märgendatud fail Tartu_V6nnu_Ahja_id18082_1885a.json
72. Märgendatud fail J2rva_Tyri_S2revere_id13610_1883a.json
73. Märgendatud fail Viljandi_K6pu_Suure-K6pu_id12977_1884a.json
74. Märgendatud fail L22ne_Kullamaa_Reop2_id15839_1868a.json
75. Märgendatud fail Tartu_N6o_Pangodi_id4217_1889a.json
76. Märgendatud fail P2rnu_Halliste_Penuja_id815_1885a.json
77. Märgendatud fail Tartu_Torma_Avinurme_id21177_1871a.json
78. Märgendatud fail L22ne_Kullamaa_Piirsalu_id16751_1889a.json
79. Märgendatud fail J2rva_Tyri_V22tsa_id18538_1894a.json
80. Märgendatud fail V6ru_R6uge_Saaluse_id11478_1879a.json
81. Märgendatud fail Tartu_Kodavere_Pala_id22839_1872a.json
82. Märgendatud fail Saare_Kihelkonna_Lymanda_id790_1868a.json
83. Märgendatud fail Harju_Rapla_Rapla_id20935_1870a.json
84. Märgendatud fail Tartu_V6nnu_Ahja_i

202. Märgendatud fail Viljandi_Kolga-Jaani_Paenasti_id330_1874a.json
203. Märgendatud fail Tartu_Torma_Avinurme_id17385_1871a.json
204. Märgendatud fail V6ru_Vastseliina_Misso_id18830_1882a.json
205. Märgendatud fail Tartu_Torma_Avinurme_id4086_1858a.json
206. Märgendatud fail J2rva_Tyri_S2revere_id10947_1871a.json
207. Märgendatud fail Harju_Kose_Habaja_id735_1874a.json
208. Märgendatud fail Viru_Haljala_Vihula_id10570_1867a.json
209. Märgendatud fail Harju_Kose_Triigi_id9690_1869a.json
210. Märgendatud fail Tartu_Kodavere_Alatskivi_id10323_1880a.json
211. Märgendatud fail V6ru_Vastseliina_Misso_id21200_1882a.json
212. Märgendatud fail J2rva_Tyri_Kirna_id23696_1873a.json
213. Märgendatud fail J2rva_J2rva-Jaani_Einmanni_id10155_1868a.json
214. Märgendatud fail J2rva_Tyri_V22tsa_id20184_1899a.json
215. Märgendatud fail Tartu_Kodavere_Alatskivi_id6197_1879a.json
216. Märgendatud fail P2rnu_T6stamaa_Kihnu_id291_1860a.json
217. Märgendatud fail Tartu_N6o_Pangodi_id5252_1889a.json
218. Märg

***** Epoch #21 *****
Loss: 6489.759332
Improvement ratio: 0.302106
Feature L2-norm: 81.422021
Learning rate (eta): 0.024974
Total number of feature updates: 340221
Seconds required for this iteration: 0.203

***** Epoch #22 *****
Loss: 6355.239784
Improvement ratio: 0.282484
Feature L2-norm: 82.859815
Learning rate (eta): 0.024973
Total number of feature updates: 356422
Seconds required for this iteration: 0.204

***** Epoch #23 *****
Loss: 6157.188518
Improvement ratio: 0.285495
Feature L2-norm: 84.268152
Learning rate (eta): 0.024971
Total number of feature updates: 372623
Seconds required for this iteration: 0.203

***** Epoch #24 *****
Loss: 6042.190924
Improvement ratio: 0.262364
Feature L2-norm: 85.640855
Learning rate (eta): 0.024970
Total number of feature updates: 388824
Seconds required for this iteration: 0.202

***** Epoch #25 *****
Loss: 5932.093531
Improvement ratio: 0.250380
Feature L2-norm: 86.989343
Learning rate (eta): 0.024969
Total number of feature updates: 405025

...

105. Märgendatud fail Tartu_V6nnu_Ahja_id14796_1882a.json
106. Märgendatud fail V6ru_R2pina_R2pina_id1168_1863a.json
107. Märgendatud fail V6ru_R6uge_Saaluse_id9577_1878a.json
108. Märgendatud fail V6ru_R2pina_Kahkva_id5750_1887a.json
109. Märgendatud fail J2rva_Tyri_Tyri-Alliku_id1370_1894a.json
110. Märgendatud fail V6ru_R2pina_Kahkva_id19205_1866a.json
111. Märgendatud fail J2rva_Tyri_S2revere_id13737_1884a.json
112. Märgendatud fail Saare_Kihelkonna_Kotlandi_id20759_1867a.json
113. Märgendatud fail L22ne_Pyhalepa_K2rdla_id22767_1869a.json
114. Märgendatud fail Tartu_Kodavere_Alatskivi_id4069_1879a.json
115. Märgendatud fail J2rva_Tyri_V22tsa_id16522_1885a.json
116. Märgendatud fail J2rva_Ambla_Ambla_id6971_1884a.json
117. Märgendatud fail L22ne_Pyhalepa_K2rdla_id25541_1878a.json
118. Märgendatud fail Harju_Kose_Palvere_id25472_1873a.json
119. Märgendatud fail V6ru_Vastseliina_Misso_id20265_1882a.json
120. Märgendatud fail J2rva_Tyri_V22tsa_id18003_1888a.json
121. Märgendatud fail T

240. Märgendatud fail Tartu_Torma_Avinurme_id17101_1871a.json
241. Märgendatud fail Harju_Kose_Palvere_id22520_1886a.json
242. Märgendatud fail J2rva_Tyri_S2revere_id8279_1885a.json
243. Märgendatud fail Tartu_V6nnu_Ahja_id17669_1885a.json
244. Märgendatud fail Tartu_Kodavere_Pala_id24085_1873a.json
245. Märgendatud fail Tartu_Torma_Avinurme_id14737_1905a.json
246. Märgendatud fail L22ne_Kullamaa_Piirsalu_id13504_1899a.json
247. Märgendatud fail L22ne_Hanila_Massu_id446_1913a.json
248. Märgendatud fail J2rva_Tyri_V22tsa_id18591_1895a.json
249. Märgendatud fail Harju_Hageri_Kohila_id21266_1885a.json
250. Märgendatud fail Tartu_R6ngu_Aakre_id5921_1890a.json
Failid märgendatud 153.09388971328735 sekundiga.
(!) Valmistan ette treenimistekste
Treenimistekstid ette valmistatud 102.46574020385742 sekundiga.
(!) Treenin NerTaggerit
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.00

***** Epoch #34 *****
Loss: 3900.784763
Improvement ratio: 0.179184
Feature L2-norm: 129.459103
Learning rate (eta): 0.049831
Total number of feature updates: 530876
Seconds required for this iteration: 0.196

***** Epoch #35 *****
Loss: 3832.959756
Improvement ratio: 0.172004
Feature L2-norm: 130.785592
Learning rate (eta): 0.049826
Total number of feature updates: 546490
Seconds required for this iteration: 0.196

***** Epoch #36 *****
Loss: 3802.500014
Improvement ratio: 0.162083
Feature L2-norm: 132.077644
Learning rate (eta): 0.049821
Total number of feature updates: 562104
Seconds required for this iteration: 0.197

***** Epoch #37 *****
Loss: 3773.590323
Improvement ratio: 0.144829
Feature L2-norm: 133.349533
Learning rate (eta): 0.049816
Total number of feature updates: 577718
Seconds required for this iteration: 0.197

***** Epoch #38 *****
Loss: 3687.929639
Improvement ratio: 0.233572
Feature L2-norm: 134.597529
Learning rate (eta): 0.049811
Total number of feature updates: 5

78. Märgendatud fail L22ne_Hanila_Massu_id361_1891a.json
79. Märgendatud fail Tartu_V6nnu_Kiidj2rve_id24846_1867a.json
80. Märgendatud fail Saare_P8ide_Laimjala_id8967_1915a.json
81. Märgendatud fail V6ru_Vastseliina_Misso_id20237_1882a.json
82. Märgendatud fail Harju_Hageri_Kohila_id11175_1875a.json
83. Märgendatud fail Harju_Hageri_Kohila_id23811_1873a.json
84. Märgendatud fail Tartu_V6nnu_Ahja_id19411_1887a.json
85. Märgendatud fail Tartu_Kodavere_Pala_id22605_1871a.json
86. Märgendatud fail Harju_Hageri_Kohila_id22513_1867a.json
87. Märgendatud fail Harju_Hageri_Kohila_id11216_1875a.json
88. Märgendatud fail Viljandi_K6pu_Suure-K6pu_id12875_1884a.json
89. Märgendatud fail Harju_Harju-Madise_Padise-Kloostri_id3009_1868a.json
90. Märgendatud fail Tartu_Kodavere_Pala_id21335_1869a.json
91. Märgendatud fail Viljandi_Pilistvere_Arussaare_id23692_1850a.json
92. Märgendatud fail Harju_J6el2htme_J6el2htme_id1558_1867a.json
93. Märgendatud fail Tartu_Kodavere_Ranna_id15169_1864a.json
94. Mä

212. Märgendatud fail J2rva_Tyri_S2revere_id16068_1889a.json
213. Märgendatud fail Tartu_Kodavere_Pala_id23072_1872a.json
214. Märgendatud fail Harju_Kose_Triigi_id11698_1876a.json
215. Märgendatud fail V6ru_P6lva_Peri_id10140_1891a.json
216. Märgendatud fail Viljandi_K6pu_Suure-K6pu_id12747_1884a.json
217. Märgendatud fail J2rva_Tyri_S2revere_id12373_1877a.json
218. Märgendatud fail Viljandi_Pilistvere_K6o_id24950_1843a.json
219. Märgendatud fail J2rva_Tyri_Kirna_id24928_1881a.json
220. Märgendatud fail P2rnu_Audru_V6lla_id5153_1877a.json
221. Märgendatud fail J2rva_Ambla_Uudekyla_id13881_1867a.json
222. Märgendatud fail Tartu_Kodavere_Alatskivi_id12243_1856a.json
223. Märgendatud fail J2rva_Tyri_S2revere_id13997_1886a.json
224. Märgendatud fail Tartu_V6nnu_Ahja_id21303_1889a.json
225. Märgendatud fail J2rva_Tyri_Kirna_id22874_1869a.json
226. Märgendatud fail V6ru_Vastseliina_Misso_id23231_1886a.json
227. Märgendatud fail Viljandi_Pilistvere_Arussaare_id23597_1845a.json
228. Märgendat